In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import _pickle as pickle

np.random.seed(20160703)
tf.set_random_seed(20160703)

In [5]:
with open('ORENIST.data', 'rb') as file:
    images, labels = pickle.load(file, encoding='latin1')

In [6]:
def edge_filter():
    filter0 = np.array(
        [[ 2, 1, 0, -1, -2],
         [ 3, 2, 0, -2, -3],
         [ 4, 3, 0, -2, -4],
         [ 3, 2, 0, -2, -3],
         [ 2, 1, 0, -1, -2]]) / 23.0 #ブロードキャストルールで全体を/23.0
    
    filter1 = np.array(
        [[ 2, 3, 4, 3, 2],
         [ 1, 2, 3, 2, 1],
         [ 0, 0, 0, 0, 0],
         [-1,-2,0,-2,-1],
         [-2,-3,-4,-3,-2]]) / 23.0
    
    filter_array = np.zeros([5, 5, 1, 2])#フィルターの情報を格納しておく多次元リスト 5x5x1x2(フィルターサイズx入力レイヤー数x出力レイヤー数)
    filter_array[:,:,0,0] = filter0 #一番目の出力レイヤーのフィルターサイズの部分に格納
    filter_array[:,:,0,1] = filter1 #2番目の...
        
    return tf.constant(filter_array, dtype=tf.float32) # 用意した多次元リストをTensorFlowの定数値オブジェクトに変換したものを返している。
        

In [7]:
x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1, 28, 28, 1])

W_conv = edge_filter()
h_conv = tf.abs(tf.nn.conv2d(x_image, W_conv, strides = [1,1,1,1], padding='SAME'))

h_conv_cutoff = tf.nn.relu(h_conv-0.2)

h_pool = tf.nn.max_pool(h_conv_cutoff, ksize = [1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [8]:
h_pool_flat = tf.reshape(h_pool, [-1, 392])

num_units1 = 392
num_units2 = 2

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.zeros([num_units2]))
hidden2 = tf.nn.tanh(tf.matmul(h_pool_flat, w2) + b2)

w0 = tf.Variable(tf.zeros([num_units2, 3]))
b0 = tf.Variable(tf.zeros([3]))
p = tf.nn.softmax(tf.matmul(hidden2, w0) +b0)
